In [2]:
# ---- Reproducibility
import random
import torch
import sys
import os
import matplotlib.pyplot as plt
from cebmf_torch import cEBMF
#seed = 1234
#random.seed(seed)
#torch.manual_seed(seed)
#if torch.cuda.is_available():
#    torch.cuda.manual_seed_all(seed)


In [3]:


# ---- Parameters
n, p = 50, 40
noise_std = 0.1

# (Optional) choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---- Data generation (PyTorch)
# Use float64 to mirror NumPy defaults
u = torch.rand(n, dtype=torch.float64, device=device)          # length n
v = torch.rand(p, dtype=torch.float64, device=device)          # length p

# Rank-1 matrix via outer product
rank_1_matrix = torch.outer(u, v)                              # (n, p)

# Homoscedastic Gaussian noise
noise = noise_std * torch.randn(n, p, dtype=torch.float64, device=device)

noisy_matrix = rank_1_matrix + noise

# ---- Print (move to CPU for readability if needed)
print("Rank-1 Matrix (Outer Product):")
print(rank_1_matrix.cpu().numpy())

print("\nNoisy Matrix (with Homoscedastic Noise):")
print(noisy_matrix.cpu().numpy())


Rank-1 Matrix (Outer Product):
[[0.28915743 0.09213396 0.35769013 ... 0.03979447 0.12895387 0.04006715]
 [0.48921275 0.15587741 0.60516022 ... 0.06732651 0.21817139 0.06778785]
 [0.5803861  0.18492789 0.71794241 ... 0.07987399 0.25883145 0.0804213 ]
 ...
 [0.53052616 0.16904106 0.65626525 ... 0.07301215 0.2365957  0.07351245]
 [0.29936931 0.09538777 0.37032231 ... 0.04119985 0.13350801 0.04148216]
 [0.49882973 0.15894166 0.61705651 ... 0.06865002 0.22246022 0.06912043]]

Noisy Matrix (with Homoscedastic Noise):
[[ 0.18715048  0.13683926  0.34348125 ... -0.00386853  0.19247289
   0.07502474]
 [ 0.52193467  0.22771712  0.50512986 ...  0.17472207  0.19230235
   0.26043551]
 [ 0.4785419   0.25428278  0.87488213 ... -0.02601621  0.38762804
   0.12603815]
 ...
 [ 0.61109474  0.32999699  0.50051302 ...  0.0012181   0.30546258
   0.18430159]
 [ 0.15930202 -0.00883887  0.33373319 ... -0.01966066  0.10964651
  -0.09302696]
 [ 0.60016656  0.14030654  0.57636693 ...  0.04301479  0.24739849
  -0.07

In [4]:
mycebmf=  cEBMF(data= noisy_matrix) 
mycebmf.initialize()
print(mycebmf.L[:,1])
print(mycebmf.F[:,1])

tensor([ 0.1240, -0.2149, -0.0410, -0.3877, -0.2755,  0.2281, -0.1114, -0.3219,
         0.2945, -0.1123, -0.1232,  0.1387,  0.0478, -0.2190,  0.0192, -0.0589,
        -0.0051, -0.1646, -0.0535, -0.0666, -0.1248, -0.3188, -0.0734, -0.1220,
         0.3127, -0.0605,  0.3836,  0.2007,  0.0073,  0.1812, -0.1719, -0.1172,
         0.1402,  0.1460,  0.1394,  0.2210,  0.0938,  0.0174,  0.0573,  0.2568,
         0.1158, -0.0293,  0.1898, -0.0880, -0.1410,  0.1319,  0.0541, -0.0869,
         0.1575, -0.3047])
tensor([-0.2431,  0.0227,  0.1310,  0.1155, -0.1013,  0.2668, -0.0760,  0.0736,
         0.2138, -0.0299,  0.0891, -0.1209,  0.0914,  0.1256, -0.0154,  0.2726,
         0.0953, -0.1832,  0.0556, -0.1858, -0.2996,  0.0595,  0.0110, -0.1911,
         0.2050, -0.2289, -0.0930, -0.1708,  0.0639,  0.0008,  0.1237, -0.0474,
        -0.2476, -0.0806, -0.1824,  0.2265,  0.1521,  0.0231, -0.3243, -0.0643])


In [ ]:
mycebmf.iter_once()
mycebmf.update_fitted_value()
import numpy as np
K=0
mycebmf.Y_fit 

In [ ]:
k=0

Rk = self._partial_residual_masked(k)  # (N,P), zeros where missing
fk = self.F[:, k]  # (P,)
fk2 = self.F2[:, k]  # (P,)

if tau_map is None:
        denom_l = (
                    (fk2.view(1, -1) * self.mask).sum(dim=1).clamp_min(eps)
                )  # (N,)
                num_l = Rk @ fk  # (N,)
                se_l = torch.sqrt(1.0 / (self.tau * denom_l))
    else:
                denom_l = (
                    (tau_map * (fk2.view(1, -1) * self.mask)).sum(dim=1).clamp_min(eps)
                )  # (N,)
                num_l = (tau_map * Rk) @ fk  # (N,)
                se_l = torch.sqrt(1.0 / denom_l)

            lhat = num_l / denom_l
        # print(denom_l)
 
X_model = self.update_cov_L(k)
with torch.enable_grad():
            resL = self.prior_L_fn(
                X=X_model,
                betahat=lhat,
                sebetahat=se_l,
                model_param=self.model_state_L[k],
            )
with torch.no_grad():
            self.model_state_L[k] = resL.model_param
            self.L[:, k] = resL.post_mean
            self.L2[:, k] = resL.post_mean2
            nm_ll_L = normal_means_loglik(
                x=lhat, s=se_l, Et=resL.post_mean, Et2=resL.post_mean2
            )
            self.kl_l[k] = torch.as_tensor((-resL.loss) - nm_ll_L, device=self.device)
            self.pi0_L[k] = resL.pi0_null if hasattr(resL, "pi0_null") else None

            # ---------- Update F[:, k] ----------
Rk = self._partial_residual_masked(k)  # recompute with updated L
lk = self.L[:, k]  # (N,)
lk2 = self.L2[:, k]  # (N,)

if tau_map is None:
                denom_f = (
                    (lk2.view(-1, 1) * self.mask).sum(dim=0).clamp_min(eps)
                )  # (P,)
                num_f = Rk.T @ lk  # (P,)
                se_f = torch.sqrt(1.0 / (self.tau * denom_f))
else:
                denom_f = (
                    (tau_map * (lk2.view(-1, 1) * self.mask)).sum(dim=0).clamp_min(eps)
                )  # (P,)
                num_f = (tau_map * Rk).T @ lk  # (P,)
                se_f = torch.sqrt(1.0 / denom_f)

            fhat = num_f / denom_f

X_model = self.update_cov_F(k)
with torch.enable_grad():
            resF = self.prior_F_fn(
                X=X_model,
                betahat=fhat,
                sebetahat=se_f,
                model_param=self.model_state_F[k],
            )
with torch.no_grad():
            self.model_state_F[k] = resF.model_param
            self.F[:, k] = resF.post_mean
            self.F2[:, k] = resF.post_mean2
            # store as scalar on device; PriorResult.loss already = -log_lik
            nm_ll_F = normal_means_loglik(
                x=fhat, s=se_f, Et=resF.post_mean, Et2=resF.post_mean2
            )
            self.kl_f[k] = torch.as_tensor((-resF.loss) - nm_ll_F, device=self.device)
            self.pi0_F[k] = resF.pi0_null if hasattr(resF, "pi0_null") else None